# Clean Data

## Story
Show US average of <b>income, debt, net price</b> for schools over the years. Then, allow for users to see how their schools compare.

## Goal
* Clean three variables
* Find the averages for each
* Store data in JSON
    * (school_name, income, debt, net_price)
    * average will have school_name == 'avg'

In [70]:
%matplotlib inline

import matplotlib.pyplot as plt
import sqlite3
import pandas as pd
import seaborn as sns

sns.set_style("white")

conn = sqlite3.connect('../data/output/database.sqlite')
c = conn.cursor()

def execute(sql):
    '''Executes a SQL command on the 'c' cursor and returns the results'''
    c.execute(sql)
    return c.fetchall()

def printByYear(data):
    '''Given a list of tuples with (year, data), prints the data next to corresponding year'''
    for datum in data:
        print "{0}: {1}".format(datum[0], datum[1])
        
years = {1996:1.46, 1997:1.43, 1998:1.4, 1999:1.38, 2000:1.33, 2001:1.3, 2002:1.28, 2003:1.25, 
         2004:1.22, 2005:1.18, 2006:1.14, 2007:1.11, 2008:1.07, 2009:1.07, 2010:1.05, 2011:1.02, 2012:1, 2013:1}

def adjustForInflation(value, year):
    '''Adjust the dollar value based on year
    Source (http://www.bls.gov/data/inflation_calculator.htm)
    '''
    if value == None:
        return
    return int(value * years[year])

In [40]:
query = """SELECT INSTNM, YEAR, 
                TUITIONFEE_IN, TUITIONFEE_OUT,
                DEBT_MDN, 
                mn_earn_wne_p10, md_earn_wne_p10, pct10_earn_wne_p10, pct25_earn_wne_p10, 
                pct75_earn_wne_p10, pct90_earn_wne_p10
            FROM Scorecard
            WHERE MAIN='Main campus'
              AND PREDDEG = 'Predominantly bachelor''s-degree granting'
        """
rawData = execute(query)

query = """SELECT INSTNM
            FROM Scorecard
            WHERE MAIN='Main campus'
              AND PREDDEG = 'Predominantly bachelor''s-degree granting'
              and CCBASIC NOT LIKE '%Special%'"""
institutions = execute(query)

Now that I have the raw data I want to structure it in a JSON like fashion. instnm -> year -> data

In [75]:
treeStruct = {} # Inflation adjusted JSON structured tree about different institutions

# Get main campus, predominantly bachelor's, non-medical schools that had data for 2013
for inst in institutions:
    treeStruct[inst[0]] = []

def setValue(data, key, value, year):
    if value and value != 'PrivacySuppressed':
        data[key] = adjustForInflation(value, year)
    
for row in rawData:
    instnm = row[0]
    year = row[1]
    
    if instnm in treeStruct:
        financialData = {}
        
        setValue(financialData, 'tuition_in', row[2], year)
        setValue(financialData, 'tuition_out', row[3], year)
        setValue(financialData, 'debt_mdn', row[4], year)
        setValue(financialData, 'mn_earn', row[5], year)
        setValue(financialData, 'md_earn', row[6], year)
        setValue(financialData, 'md_earn_10', row[7], year)
        setValue(financialData, 'md_earn_25', row[8], year)
        setValue(financialData, 'md_earn_75', row[9], year)
        setValue(financialData, 'md_earn_90', row[10], year)

        if len(financialData) != 0:
            treeStruct[instnm].append({year: financialData})
            


In [76]:
treeStruct['Harvard University']

[{2005: {'debt_mdn': 9263, 'tuition_in': 37874, 'tuition_out': 37874}},
 {2006: {'debt_mdn': 9329, 'tuition_in': 38428, 'tuition_out': 38428}},
 {2007: {'debt_mdn': 9610,
   'md_earn': 96570,
   'md_earn_10': 24198,
   'md_earn_25': 55611,
   'md_earn_75': 199467,
   'md_earn_90': 277500,
   'mn_earn': 164724,
   'tuition_in': 38847,
   'tuition_out': 38847}},
 {2008: {'debt_mdn': 6420, 'tuition_in': 38705, 'tuition_out': 38705}},
 {2009: {'debt_mdn': 6420,
   'md_earn': 82176,
   'md_earn_10': 20116,
   'md_earn_25': 49755,
   'md_earn_75': 169167,
   'md_earn_90': 257228,
   'mn_earn': 128721,
   'tuition_in': 39602,
   'tuition_out': 39602}},
 {2010: {'debt_mdn': 6300, 'tuition_in': 40335, 'tuition_out': 40335}},
 {2011: {'debt_mdn': 6120,
   'md_earn': 88944,
   'md_earn_10': 25194,
   'md_earn_25': 52632,
   'md_earn_75': 173502,
   'md_earn_90': 255000,
   'mn_earn': 133110,
   'tuition_in': 40648,
   'tuition_out': 40648}},
 {2012: {'debt_mdn': 6250, 'tuition_in': 39966, 'tuitio